In [1]:
import os,sys
from pathlib import Path
currentdir = Path(os.getcwd())
sys.path.insert(0, os.path.dirname(currentdir.parent))
sys.path.insert(0, os.path.dirname(currentdir))
print(str(sys.path[0:3]))

['d:\\BAMT', 'd:\\', 'd:\\BAMT\\tutorials']


In [3]:
from bamt.networks.hybrid_bn import HybridBN
import bamt.preprocessors as pp

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from pgmpy.estimators import K2Score

In [21]:
data = pd.read_csv(r'../Data/real data/vk_data.csv')
data

,sex,age,city_id,has_high_education,relation,num_of_relatives,followers_count,status,mobile_phone,twitter,...,"car, cars, sale, tires","sale, clothes, size, price, sewing","ingredients, cooking, dishes, salt, spices","theater, performance, actors","photographer, photography, show, wedding, photographer",top1_interes,top2_interes,top3_interes,top4_interes,top5_interes
0,1,50.0,2,0,0,2,27,0,0,0,...,0.201622,3.397819e-02,0.003674,3.081348e-01,1.070007e-02,"theater, performance, actors","car, cars, sale, tires","participation, festival, exhibition","client, project, internet, business, company","home, renovation, decoration, installation"
1,2,36.0,2,0,0,2,6,0,0,0,...,0.060985,2.529792e-02,0.020964,1.153278e-02,3.497006e-03,"book, reading, language, word, literature","music, concerts, rock, tickets, festivals","school, education, question, topic","client, project, internet, business, company","war, history, army, politics"
2,1,65.0,2,0,0,2,25,0,0,0,...,0.000009,2.007164e-09,0.100547,1.566668e-03,4.916993e-02,"client, project, internet, business, company","skin, face, care, cosmitology","ingredients, cooking, dishes, salt, spices","treatment, doctors, tests, clinic, health","photographer, photography, show, wedding, phot..."
3,2,30.0,2,0,0,2,52,0,1,0,...,0.143803,5.324164e-03,0.027627,1.481971e-02,1.903577e-02,"war, history, army, politics","car, cars, sale, tires","film, series, cinema, genre","music, concerts, rock, tickets, festivals","football, match, russia, team, goalkeeper"
4,2,36.0,2,0,0,2,18,0,0,0,...,0.037360,4.527913e-02,0.087596,4.388889e-02,7.207429e-02,"home, renovation, decoration, installation","ingredients, cooking, dishes, salt, spices","game, team, competition, sport","massage, meditation, yoga","shop, price, gifts, internet"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,2,50.0,2,0,0,2,24,0,1,0,...,0.021852,1.240507e-02,0.000604,2.096222e-02,6.373252e-03,"music, concerts, rock, tickets, festivals","skin, face, care, cosmitology","film, series, cinema, genre","handmade, materials, products","client, project, internet, business, company"
29996,1,37.0,2,0,0,2,0,4,0,0,...,0.010729,2.396505e-02,0.085240,3.546863e-02,3.498729e-02,"participation, festival, exhibition","ingredients, cooking, dishes, salt, spices","shop, price, gifts, internet","beauty, hair, manicure, salon","game, team, competition, sport"
29997,2,55.0,2,0,4,60,4,0,1,0,...,0.000000,0.000000e+00,0.001092,8.027095e-09,3.005514e-10,"music, concerts, rock, tickets, festivals","color, flowers, interior, design","book, reading, language, word, literature","ingredients, cooking, dishes, salt, spices","game, team, competition, sport"
29998,1,39.0,2,0,0,2,102,0,0,0,...,0.013108,2.857535e-02,0.127433,7.420912e-02,3.401194e-02,"ingredients, cooking, dishes, salt, spices","vacation, sea, accommodation, cost","home, renovation, decoration, installation","theater, performance, actors","game, team, competition, sport"


In [22]:
cols = ['age', 'sex', 'has_pets', 'is_parent', 'relation', 'is_driver', 'tr_per_month', 'median_tr', 'mean_tr']
data = data[cols]
data[['sex',	'has_pets',	'is_parent',	'relation',	'is_driver']] = data[['sex',	'has_pets',	'is_parent',	'relation',	'is_driver']].astype(str)

In [23]:
data

,age,sex,has_pets,is_parent,relation,is_driver,tr_per_month,median_tr,mean_tr
0,50.0,1,0,0,0,0,46.727033,271.500,1032.251417
1,36.0,2,0,0,0,0,0.421272,2000.000,6518.500000
2,65.0,1,0,0,0,0,6.087375,1751.025,11006.916714
3,30.0,2,1,1,0,0,39.550742,799.500,1821.201565
4,36.0,2,0,0,0,0,18.719393,660.000,2319.134911
...,...,...,...,...,...,...,...,...,...
29995,50.0,2,0,0,0,0,7.222309,4250.000,7077.928571
29996,37.0,1,0,1,0,0,1.407486,975.855,1105.408750
29997,55.0,2,0,0,4,0,18.749115,751.730,1543.378831
29998,39.0,1,0,0,0,0,1.000000,18000.000,18000.000000


In [24]:
encoder = preprocessing.LabelEncoder()
discretizer = preprocessing.KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='quantile')

p = pp.Preprocessor([('encoder', encoder), ('discretizer', discretizer)])
discretized_data, est = p.apply(data)

In [25]:
bn = HybridBN(has_logit=True, use_mixture=True) # init BN
info = p.info
info

{'types': {'age': 'cont',
  'sex': 'disc',
  'has_pets': 'disc',
  'is_parent': 'disc',
  'relation': 'disc',
  'is_driver': 'disc',
  'tr_per_month': 'cont',
  'median_tr': 'cont',
  'mean_tr': 'cont'},
 'signs': {'age': 'pos',
  'tr_per_month': 'pos',
  'median_tr': 'pos',
  'mean_tr': 'pos'}}

In [26]:
bn.add_nodes(info)
bn.add_edges(discretized_data,  scoring_function=('K2',K2Score))

  0%|          | 19/1000000 [00:00<13:27:08, 20.65it/s]


In [27]:
bn.get_info()

,name,node_type,data_type,parents,parents_types
0,relation,Discrete,disc,[],[]
1,is_driver,Discrete,disc,[],[]
2,is_parent,Discrete,disc,[relation],[disc]
3,sex,Discrete,disc,"[is_parent, relation, is_driver]","[disc, disc, disc]"
4,age,ConditionalMixtureGaussian,cont,"[sex, is_parent, relation]","[disc, disc, disc]"
5,median_tr,ConditionalMixtureGaussian,cont,"[age, sex, is_parent]","[cont, disc, disc]"
6,mean_tr,ConditionalMixtureGaussian,cont,"[age, median_tr, is_parent]","[cont, cont, disc]"
7,tr_per_month,ConditionalMixtureGaussian,cont,"[median_tr, mean_tr, sex, is_parent]","[cont, cont, disc, disc]"
8,has_pets,ConditionalLogit (LogisticRegression),disc,"[tr_per_month, sex]","[cont, disc]"


In [28]:
bn.fit_parameters(data)

In [29]:
test = data[['age', 'sex', 'is_parent', 'relation', 'is_driver', 'tr_per_month', 'median_tr', 'mean_tr', 'has_pets']].sample(100)
target = test['has_pets'].values
test = test.drop(columns=['has_pets'])
test.reset_index(inplace=True, drop=True)
test

,age,sex,is_parent,relation,is_driver,tr_per_month,median_tr,mean_tr
0,23.0,1,0,0,0,7.745763,318.00,524.121474
1,25.0,2,0,0,0,16.937675,250.00,969.350602
2,36.0,1,1,0,0,4.150483,1237.00,1678.961111
3,34.0,2,0,0,0,1.000000,17011.18,17011.180000
4,40.0,2,0,1,0,6.621763,2931.00,21269.454912
...,...,...,...,...,...,...,...,...
95,27.0,2,0,0,0,11.192915,129.81,426.930175
96,46.0,2,0,4,0,2.560563,7000.00,17161.740755
97,39.0,1,1,0,0,5.064773,1050.00,2994.389810
98,26.0,2,0,0,0,37.098761,167.00,1506.660675


In [30]:
pred = bn.predict(test, 5)

100%|██████████| 100/100 [00:03<00:00, 28.25it/s]


In [31]:
accuracy_score(target, pred['has_pets'])

0.98